In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
def create_test_prompt(topic, num_questions, num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions} "\
            +f"Each question should have {num_possible_answers} options. "\
            +f"Also include the correct answer for each question using the starting string 'Correct Answer:'"
    return prompt

print(create_test_prompt("topicA", 2, 3))

Create a multiple choice quiz on the topic of topicA consisting of 2 Each question should have 3 options. Also include the correct answer for each question using the starting string 'Correct Answer:'


In [5]:
print(create_test_prompt("Python", 4, 4))

Create a multiple choice quiz on the topic of Python consisting of 4 Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct Answer:'


In [10]:
response = openai.Completion.create(engine='text-davinci-003',
                                    prompt = create_test_prompt('US History', 4, 4),
                                    max_tokens = 256,
                                    temperature = 0.7
                                   )

In [14]:
response['choices'][0]['text']

'\n\nQ1: When was the Declaration of Independence signed?\nA. 1776\nB. 1775\nC. 1777\nD. 1778\nCorrect Answer: A. 1776\n\nQ2: Who was the first president of the United States?\nA. George Washington\nB. John Adams\nC. Thomas Jefferson\nD. James Madison\nCorrect Answer: A. George Washington\n\nQ3: Who wrote the Federalist Papers?\nA. Thomas Jefferson\nB. James Madison\nC. Alexander Hamilton\nD. John Adams\nCorrect Answer: C. Alexander Hamilton\n\nQ4: Which amendment gave women the right to vote?\nA. 14th Amendment\nB. 15th Amendment\nC. 16th Amendment\nD. 19th Amendment\nCorrect Answer: D. 19th Amendment'

In [28]:
def create_student_view(test, num_questions):
    student_view = {1: ''}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+'\n'
        else:
            if question_number <  num_questions:
                question_number +=1
                student_view[question_number] = ''
    return student_view

In [30]:
student_view = create_student_view(response['choices'][0]['text'], 4)

In [31]:
print(student_view.get(1))



Q1: When was the Declaration of Independence signed?
A. 1776
B. 1775
C. 1777
D. 1778



In [47]:
def extract_answer(test, num_questions):
    answers = {1: ''}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += line+'\n'
            if question_number < num_questions:
                 question_number +=1
                 answers[question_number] = ''
    return answers

In [50]:
answers = extract_answer(response['choices'][0]['text'], 4)

In [42]:
# Take The Exam
def take(student_view):
    student_answers = {}
    for question, question_view in student_view.items():
        print (question_view)
        answer = input("Enter your answer: ")
        student_answers[question] = answer
    return student_answers

In [43]:
student_answers = take(student_view)



Q1: When was the Declaration of Independence signed?
A. 1776
B. 1775
C. 1777
D. 1778

Enter your answer: A

Q2: Who was the first president of the United States?
A. George Washington
B. John Adams
C. Thomas Jefferson
D. James Madison

Enter your answer: B

Q3: Who wrote the Federalist Papers?
A. Thomas Jefferson
B. James Madison
C. Alexander Hamilton
D. John Adams

Enter your answer: C

Q4: Which amendment gave women the right to vote?
A. 14th Amendment
B. 15th Amendment
C. 16th Amendment
D. 19th Amendment

Enter your answer: D


In [44]:
student_answers

{1: 'A', 2: 'B', 3: 'C', 4: 'D'}

In [51]:
answers

{1: 'Correct Answer: A. 1776\n',
 2: 'Correct Answer: A. George Washington\n',
 3: 'Correct Answer: C. Alexander Hamilton\n',
 4: 'Correct Answer: D. 19th Amendment\n'}

In [57]:
def grade(correct_answer_dict, student_answers):
    correct_answers = 0
    for question, answer in student_answers.items():
        if answer.upper() == correct_answer_dict[question][16]:
            correct_answers +=1
    grade = 100*correct_answers / len(answers)
    
    if grade < 60 :
        passed = "NO PASS"
    else:
        passed = "PASS"
    
    return F"{correct_answers}/{len(answers)} correct! You got {grade} grade, {passed}"

In [58]:
grade(answers, student_answers)

'3/4 correct! You got 75.0 grade, PASS'